In [3]:
"""
Scrape Ghana Districts data from https://www.citypopulation.de/
"""
import bs4
import requests
from lxml import html
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set(rc={'figure.figsize':(20,16)})
#ns.set_theme(style='darkgrid')

In [4]:
#get web data

#link for web data
url="https://www.citypopulation.de/en/ghana/admin/"

res=requests.get(url)
res.raise_for_status() #check if link is valid

<h1><b>BEAUTIFULSOUP</b></h1>

In [60]:
#parse web data using bs4

data=bs4.BeautifulSoup(res.content, 'html.parser')

In [6]:
# get table columns which should be marked with table head tag 'th'

head = data.find_all('th')

columns=[]

for td in head:
    columns.append(td.text)

columns=columns[:4]
#print(columns)

In [7]:
body = data.find_all('tbody')


body_rows=[]

for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
body_rows = body_rows[:len(body_rows)-4]
#print(body_rows)

<h2><b>REGIONAL DATA</b></h2>

In [63]:
regions = []
i = 0
while i < len(body_rows):
    regions.append(body_rows[i])
    i+=2

#print(regions) 

In [9]:
regcolumns=columns.copy()
regcolumns.append('Misc')

In [65]:
df = pd.DataFrame(regions, columns=regcolumns)
df = df.drop(df[['Misc', 'Status']],axis=1)
df.columns = ['Region', 'Census', 'Projected']
df.replace(',','',regex=True,inplace=True)
df[['Census', 'Projected']]=df[['Census','Projected']].astype('int64')
df

,Region,Census,Projected
0,Ahafo (← Brong Ahafo),484210,613049
1,Ashanti,4780380,5924498
2,Bono (← Brong Ahafo),922617,1168807
3,Bono East (← Brong Ahafo),904156,1133768
4,Central,2201863,2605492
5,Eastern,2633154,3318853
6,Greater Accra,4010054,5055883
7,North East (← Northern),465005,588800
8,Northern,1544946,1948913
9,Oti (← Volta),605725,759799


<h2><b>DISTRICTS DATA</b></h2>

In [11]:
district_row = []

i = 1
while i < len(body_rows):
    district_row.append(body_rows[i])
    i+=2
#print(len(district_row))

In [12]:
district_row = []

i = 1
while i < len(body_rows):
    district_row.append(body_rows[i])
    i+=2
#print(len(district_row))

In [13]:
district = []
status = []
census = []
projected = []

In [14]:
for row in district_row:
    j=0
    while j < len(row):
        district.append(row[j])
        j+=5
#print(len(district))

In [15]:
for row in district_row:
    j=1
    while j < len(row):
        status.append(row[j])
        j+=5
#print(len(status))

In [16]:
for row in district_row:
    j=2
    while j < len(row):
        census.append(row[j])
        j+=5
#print(len(census))

In [17]:
for row in district_row:
    j=3
    while j < len(row):
        projected.append(row[j]) 
        j+=5
#print(len(projected))

In [18]:
df=pd.DataFrame(data={'District':district, 'Status':status, 'Census':census, 'Projected':projected})
df.replace(',','',regex=True,inplace=True)
df[['Census', 'Projected']]=df[['Census','Projected']].astype('int64')
df

,District,Status,Census,Projected
0,Asunafo North Municipal,Municipal District,124685,157870
1,Asunafo South,District,95580,120976
2,Asutifi North,District,52259,66141
3,Asutifi South,District,53584,67763
4,Tano North Municipal,Municipal District,79973,101305
...,...,...,...,...
255,Bodi,District,53314,71133
256,Juabeso,District,58435,77963
257,Sefwi Akontombra,District,82467,109868
258,Sefwi Wiawso Municipal,Municipal District,139200,185826


<h2><b>NATIONAL</b></h2>

In [61]:
country=data.find_all('tbody', class_="admin0")
td=country[0].find_all('td')

country=[]
for i in range(len(td)-1):
    if i>=2:
        country.append(td[i].text)
#country

In [62]:
df=pd.DataFrame([country], columns=['Total Population (2010 Census)', 'Projected Population(2020)'])
df

,Total Population (2010 Census),Projected Population(2020)
0,"24,658,823","30,955,204"


In [41]:
#scrape population distribution data for Ghana

details=data.find_all(class_='addinfo') 

In [42]:
#add distribution table values to 

data1=[]
data2=[]

for i in range(len(details)):
    section=details[i].find_all('td')
    for j in range(len(section)):
        if j%2==0:
            data1.append(section[j].text)
        else:
            data2.append(section[j].text)

In [43]:
pop_dist={}

#add the column headings and values to dictionary newdata
for col in data1:
    for pop in data2:
        pop_dist[col]=pop

In [57]:
#create dataframe from dictionary

df = pd.DataFrame([pop_dist])
#df.reset_index(drop=True, inplace=True)


#remove , from values
df=df.replace(',', '',regex=True)


#convert all value to interger
df[list(df.columns)] = df[list(df.columns)].astype('int64')
df

,Males,Females,0-14 years,15-64 years,65+ years,0-9 years,10-19 years,20-29 years,30-39 years,40-49 years,50-59 years,60-69 years,70-79 years,80+ years
0,389657,389657,389657,389657,389657,389657,389657,389657,389657,389657,389657,389657,389657,389657


In [45]:
#df = df.append(newdata, ignore_index=True)
#df

In [46]:
#create dataframe from dictionary with list values using Series

#columns=['Gender', 'Gender Pop', 'Age Group', 'Age Group Pop', 'Age Distribution', 'Age Distribution Pop' ]

#df2=pd.DataFrame({columns[0]:pd.Series(data1[0:2]), columns[1]:pd.Series(data2[0:2]), columns[2]:pd.Series(data1[2:5]),columns[3]:pd.Series(data2[2:5]), columns[4]:pd.Series(data1[5:]),columns[5]:pd.Series(data2[5:])})
#df2.fillna('')

In [47]:
#create dataframe from dictionary with list values using from_list

#df3=pd.DataFrame.from_dict({columns[0]:data1[0:2], columns[1]:data2[0:2], columns[2]:data1[2:5], 
                            #columns[3]:data2[2:5], columns[4]:data1[5:],columns[5]:data2[5:]}, orient='index').T
#df3.fillna('')

<h1><b>LXML</b></h1>

In [48]:
data=html.fromstring(res.content)

<h2><b>REGIONAL DATA</b></h2>

In [49]:
#scrape using xpath(get this from inspecting the page and copying the full xpath
regions=[] 
cen_pop=[]
proj_pop=[]
area_size=[]

r_i=1
while r_i < 32:
    
    region=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr/td[1]/a/span/text()')
    census=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr/td[3]/text()')
    projected_pop=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr/td[4]/text()')
    area=data.xpath(f'/html/body/article/table/tbody{[r_i]}/tr[2]/td/p[4]/span/text()')
    

    
    regions.append(region[0])
    cen_pop.append(census[0])
    proj_pop.append(projected_pop[0])
    area_size.append(area)
    r_i+=2

#print(regions)

In [50]:
#create dataframe from data

reg_pop = pd.DataFrame({'Region':regions, 'Census':cen_pop, 'Projected':proj_pop})
reg_pop=reg_pop.replace(',','', regex=True)
reg_pop[['Census', 'Projected']]=reg_pop[['Census', 'Projected']].astype('int64')
reg_pop

#reg_pop.to_csv('Population_by_Region_(GH).csv', index=False)

,Region,Census,Projected
0,Ahafo,484210,613049
1,Ashanti,4780380,5924498
2,Bono,922617,1168807
3,Bono East,904156,1133768
4,Central,2201863,2605492
5,Eastern,2633154,3318853
6,Greater Accra,4010054,5055883
7,North East,465005,588800
8,Northern,1544946,1948913
9,Oti,605725,759799


In [51]:
#transform the data using melt so both populations can do a grouped graph

#newreg_pop = pd.melt(reg_pop, id_vars='Region', var_name='Type', value_name='Population')

<h2><b>DISTRICTS DATA</b></h2>

In [52]:
districts=[]
dist_cen_pop=[]
dist_proj_pop=[]
status=[]

In [53]:
districts=[]
dist_cen_pop=[]
dist_proj_pop=[]
status=[]

dbi=2
while dbi<33:
    
    district=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td/a/span/text()')
    census=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td[3]/text()')
    proj_pop=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td[4]/text()')
    dist_type=data.xpath(f'/html/body/article/table/tbody{[dbi]}/tr/td[2]/text()')
    
    for dist in district:
        districts.append(dist)
    for popn1 in census:
        dist_cen_pop.append(popn1)
    for popn2 in proj_pop:
        dist_proj_pop.append(popn2)
    for type in dist_type:
        status.append(type)
    dbi+=2

In [54]:
dist_pop=pd.DataFrame(data={'District':districts, 'Status':status, 'Census':dist_cen_pop, 'Projected':dist_proj_pop})
dist_pop.replace(',','',regex=True,inplace=True)
dist_pop[['Census', 'Projected']]=dist_pop[['Census','Projected']].astype('int64')
dist_pop
#dist_pop.to_csv('Population_by_District_(GH).csv', index=False)

,District,Status,Census,Projected
0,Asunafo North Municipal,Municipal District,124685,157870
1,Asunafo South,District,95580,120976
2,Asutifi North,District,52259,66141
3,Asutifi South,District,53584,67763
4,Tano North Municipal,Municipal District,79973,101305
...,...,...,...,...
255,Bodi,District,53314,71133
256,Juabeso,District,58435,77963
257,Sefwi Akontombra,District,82467,109868
258,Sefwi Wiawso Municipal,Municipal District,139200,185826


In [56]:
country=data.xpath('/html/body/article/table/tbody[33]/tr/td[1]/text()')
census=data.xpath('/html/body/article/table/tbody[33]/tr/td[3]/text()')
projected=data.xpath('/html/body/article/table/tbody[33]/tr/td[4]/text()')